In [1]:
%load_ext autoreload
%load_ext autotime

time: 558 µs (started: 2023-07-03 08:32:52 -07:00)


In [2]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')

time: 608 µs (started: 2023-07-02 22:51:58 -07:00)


In [3]:
import dask
import dask.dataframe as dd
import json
import numpy as np
import pandas as pd
from dask import compute, delayed
from dask.distributed import Client, Future, LocalCluster, as_completed, get_client, wait
from typing import Union

time: 2.51 s (started: 2023-07-02 22:51:58 -07:00)


In [4]:
client = Client(LocalCluster(n_workers=16, local_directory=f"/var/tmp/playground-cm1-challenges"))
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 48,Total memory: 125.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33257,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 48
Started: Just now,Total memory: 125.47 GiB
Comm: tcp://127.0.0.1:34815,Total threads: 3
Dashboard: http://127.0.0.1:39671/status,Memory: 7.84 GiB
Nanny: tcp://127.0.0.1:41027,


time: 2.7 s (started: 2023-07-02 22:52:01 -07:00)


In [5]:
# log_dir = "/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
log_dir = "/p/vast1/iopp/recorder_app_logs/montage_pegasus/nodes-32/_parquet"

time: 647 µs (started: 2023-07-02 22:52:03 -07:00)


In [6]:
all_view = dd.read_parquet(f"{log_dir}/*.parquet")
all_view

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=36,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 122 ms (started: 2023-07-02 22:52:03 -07:00)


In [10]:
all_view['file_dir'] = all_view['file_name'].apply(lambda x: '/'.join(x.split('/')[:-1]))
all_view.compute()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id,file_dir
0,1,132672,0,321552,0,3,2.137661,2.137760,fopen64,0,...,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,0,0,0.000000,0.000099,21377103,4007557435994691998,2303625805466626068,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
1,2,132672,0,321552,0,1,2.137835,2.138117,fread,0,...,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,2,0,0.006764,0.000282,21379759,4007557435994691998,2303625805466626068,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
2,3,132672,0,321552,0,3,2.138125,2.138141,fclose,0,...,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,0,0,0.000000,0.000016,21381327,4007557435994691998,2303625805466626068,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
3,4,132672,0,321552,0,3,2.138159,2.138171,fopen64,0,...,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,0,0,0.000000,0.000012,21381647,4007557435994691998,2303625805466626068,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
4,5,132672,0,321552,0,1,2.138231,2.138246,fread,0,...,mProject#lassen192#132672#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,2,0,0.127157,0.000015,21382382,4007557435994691998,2303625805466626068,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286057,2383210,63830,0,321552,0,2,18.125608,18.125610,fwrite,0,...,mAdd#lassen734#63830#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,2880,0,2746.582031,0.000001,181256095,4007557436607364090,369428171584890900,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
286058,2383211,63830,0,321552,0,3,18.125628,18.126268,fclose,0,...,mAdd#lassen734#63830#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,0,0,0.000000,0.000640,181259478,4007557436607364090,369428171584890900,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
286059,2383212,63830,0,321552,0,1,18.126276,18.131586,fread,0,...,mAdd#lassen734#63830#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,2880,0,0.517237,0.005310,181289317,4007557438554618451,369428171584890900,/p/gpfs1/haridev/iopp/montage-workflow/scratch...
286060,2383213,63830,0,321552,0,2,18.131601,18.131601,fwrite,0,...,mAdd#lassen734#63830#35184372410384,/p/gpfs1/haridev/iopp/montage-workflow/scratch...,2880,0,0.000000,0.000000,181316017,4007557438554618451,369428171584890900,/p/gpfs1/haridev/iopp/montage-workflow/scratch...


time: 1min 18s (started: 2023-06-26 12:15:47 -07:00)


## Eval II: Preprocessing - Query via string vs. integer indices

In [11]:
file_grp = all_view.groupby(['file_id', 'file_name']).agg({'size': sum})
files = file_grp.index.compute()
files

MultiIndex([(-9220642067073431512,            '2-fit.000020.000022.txt'),
            (-9219592665727134543,            '2-fit.000020.000032.txt'),
            (-9208777270848206005,          '2-diff.000010.000046.fits'),
            (-9193452382356441692,            '1-fit.000025.000058.txt'),
            (-9184712547690414218,            '3-fit.000058.000062.txt'),
            (-9183407037881195326,          '2-diff.000023.000054.fits'),
            (-9176824604111592157,     '2-diff.000044.000052_area.fits'),
            (-9163837277668568389,            '2-fit.000020.000059.txt'),
            (-9162483659854499299,          '2-diff.000058.000062.fits'),
            (-9151932540562165223,            '2-fit.000028.000031.txt'),
            ...
            ( 8857004603826425559,       'cposs2ukstu_red_001_004.fits'),
            ( 8927141822935412591,  'cposs2ukstu_red_007_002_area.fits'),
            ( 8948728582963324486,  'cposs2ukstu_red_006_007_area.fits'),
            ( 89953906

time: 6.98 s (started: 2023-06-26 12:17:05 -07:00)


In [12]:
file_ids = files.get_level_values(0)
file_names = files.get_level_values(1)

time: 2.29 ms (started: 2023-06-26 12:17:12 -07:00)


In [13]:
file_id_view = all_view.set_index('file_id').compute()
file_name_view = all_view.set_index('file_name').compute()

time: 1min 30s (started: 2023-06-26 12:17:12 -07:00)


In [14]:
(19680 - (19680 % 2 ** 3)) / 2 ** 3

2460.0

time: 3.65 ms (started: 2023-06-26 12:21:43 -07:00)


In [11]:
from time import time


def m(l, nel):
    return l[:int(nel)].memory_usage(deep=True)


def x(df, l, nel):
    # print(l, nel)
    t0 = time()
    df.loc[l[0]: l[int(nel) - 1]]
    t1 = time() - t0
    # print(f"time,{t1}")
    memory_usage = m(l, nel)
    # print(f"mem,{memory_usage}")

time: 2.03 ms (started: 2023-07-02 22:53:45 -07:00)


In [53]:
mnel = (19680 - (19680 % 2 ** 3)) / 2 ** 3
nels = [mnel, mnel * 2, mnel * 4]
nels

[2460.0, 4920.0, 9840.0]

time: 4.58 ms (started: 2023-06-26 12:41:48 -07:00)


### Integer range queries

In [54]:
%%timeit -r 10 -n 1000

x(file_id_view, file_ids, nels[0])

130 µs ± 2.86 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
time: 1.3 s (started: 2023-06-26 12:41:52 -07:00)


In [55]:
m(file_ids, nels[0])

19680

time: 3.83 ms (started: 2023-06-26 12:41:54 -07:00)


In [56]:
%%timeit -r 10 -n 1000

x(file_id_view, file_ids, nels[1])

125 µs ± 2.33 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
time: 1.26 s (started: 2023-06-26 12:41:54 -07:00)


In [57]:
m(file_ids, nels[1])

39360

time: 3.62 ms (started: 2023-06-26 12:41:55 -07:00)


In [58]:
%%timeit -r 10 -n 1000

x(file_id_view, file_ids, nels[2])

132 µs ± 7.78 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
time: 1.33 s (started: 2023-06-26 12:41:55 -07:00)


In [59]:
m(file_ids, nels[2])

78720

time: 3.31 ms (started: 2023-06-26 12:41:56 -07:00)


### String range queries

In [52]:
%%timeit -r 10 -n 1000

x(file_name_view, file_names, nels[0])

691 µs ± 10.8 µs per loop (mean ± std. dev. of 100 runs, 1,000 loops each)
time: 1min 9s (started: 2023-06-26 12:36:44 -07:00)


In [37]:
m(file_names, nels[0])

220717

time: 4.54 ms (started: 2023-06-26 12:29:39 -07:00)


In [38]:
%%timeit -r 10 -n 1000

x(file_name_view, file_names, nels[1])

1.1 ms ± 7.66 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
time: 11 s (started: 2023-06-26 12:29:44 -07:00)


In [39]:
m(file_names, nels[1])

510877

time: 6.15 ms (started: 2023-06-26 12:30:02 -07:00)


In [40]:
%%timeit -r 10 -n 1000

x(file_name_view, file_names, nels[2])

1.91 ms ± 11.1 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)
time: 19.1 s (started: 2023-06-26 12:30:09 -07:00)


In [41]:
m(file_names, nels[2])

995507

time: 5.49 ms (started: 2023-06-26 12:30:29 -07:00)


In [ ]:
%%timeit -n 1 -r 5

for nel in nels:
    x(file_id_view, file_ids, nel)

In [ ]:
%%timeit -n 1 -r 5

for nel in nels:
    x(file_name_view, file_names, nel)

## Eval III: Preprocessing - Iteration vs. preprocessing

In [34]:
all_view['file_dir'] = all_view['file_name'].apply(lambda x: '/'.join(x.split('/')[:-1]))
all_view.groupby(['file_dir']).agg({'size': sum}).sort_values('size').compute()

,size
file_dir,
,0
/p/gpfs1/iopp/temp/montage,0
/p/gpfs1/iopp/temp/montage/2854964,0
/p/gpfs1/iopp/temp/montage/2854964/30,24072977
/p/gpfs1/iopp/temp/montage/2854964/7,72092250
...,...
/p/gpfs1/iopp/temp/montage/2854964/6/run,1421042712
/p/gpfs1/iopp/temp/montage/2854964/0/run,3254544638
/p/gpfs1/iopp/temp/montage/2854964/25/run,4237098833


time: 15 s (started: 2023-06-21 15:29:23 -07:00)


In [37]:
all_view['file_dir_id'] = all_view['file_id'] & 0b0111111111111111111111111111111100000000000000000000000000000000
all_view.compute()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id,file_dir,file_dir_id
0,1,111084,0,314672,0,3,0.242405,0.242414,__xstat64,0,...,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.000000,0.000009,2424094,1571755611461267775,-4245390515581891276,/p/gpfs1/iopp/temp/montage/2854964,1571755608527339520
1,2,111084,0,314672,0,3,0.242502,0.242732,fopen64,0,...,/p/gpfs1/iopp/temp/montage/2854964/2/run/Kimag...,0,0,0.000000,0.000230,2426168,7638324191939094055,-4245390515581891276,/p/gpfs1/iopp/temp/montage/2854964/2/run,7638324191741083648
2,3,111084,0,314672,0,3,0.242851,0.242868,opendir,0,...,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.000000,0.000017,2428593,1571755611461267775,-4245390515581891276,/p/gpfs1/iopp/temp/montage/2854964,1571755608527339520
3,4,111084,0,314672,0,3,0.242946,0.242956,__xstat64,0,...,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.000000,0.000010,2429507,1571755611461267775,-4245390515581891276,/p/gpfs1/iopp/temp/montage/2854964,1571755608527339520
4,5,111084,0,314672,0,3,0.243006,0.243016,__xstat64,0,...,/p/gpfs1/iopp/temp/montage/2854964,0,0,0.000000,0.000010,2430107,5347462497940582382,-4245390515581891276,/p/gpfs1/iopp/temp/montage,5347462496607272960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235304,2332457,41731,31,317888,0,2,1.604730,1.604747,fwrite,0,...,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,96920,0,5469.237793,0.000017,16047384,7686174940287124680,8238577613900339652,/p/gpfs1/iopp/temp/montage/2854964/1/run,7686174937781895168
235305,2332458,41731,31,317888,0,2,1.604971,1.604992,fwrite,0,...,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,96920,0,4359.911133,0.000021,16049814,7686174937900816169,8238577613900339652,/p/gpfs1/iopp/temp/montage/2854964/1/run,7686174937781895168
235306,2332459,41731,31,317888,0,3,1.605062,1.605241,fclose,0,...,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,0,0,0.000000,0.000179,16051513,7686174940287124680,8238577613900339652,/p/gpfs1/iopp/temp/montage/2854964/1/run,7686174937781895168
235307,2332460,41731,31,317888,0,3,1.605244,1.605379,fclose,0,...,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,0,0,0.000000,0.000135,16053112,7686174937900816169,8238577613900339652,/p/gpfs1/iopp/temp/montage/2854964/1/run,7686174937781895168


time: 54 s (started: 2023-06-21 15:33:21 -07:00)


In [35]:
all_view['file_dir_id'] = all_view['file_id'] & 0b0111111111111111111111111111111100000000000000000000000000000000
all_view.groupby(['file_dir_id']).agg({'size': sum, 'file_dir': list}).sort_values('size').compute()

,size,file_dir
file_dir_id,,
0,0,"[, , , , , , , , , , , , , , , , , , , , , , ,..."
113104467636781056,0,"[, , , , , , , , , , , , , , , , , , , , , , ,..."
1571755608527339520,0,"[/p/gpfs1/iopp/temp/montage/2854964, /p/gpfs1/..."
5347462496607272960,0,"[/p/gpfs1/iopp/temp/montage, /p/gpfs1/iopp/tem..."
8459554455933681664,24072977,"[/p/gpfs1/iopp/temp/montage/2854964/30, /p/gpf..."
...,...,...
7668160539272413184,1726748997,"[/p/gpfs1/iopp/temp/montage/2854964/29/run, /p..."
7672382663923073024,3254544638,"[/p/gpfs1/iopp/temp/montage/2854964/0/run, /p/..."
7701937536477691904,4237098833,"[/p/gpfs1/iopp/temp/montage/2854964/25/run, /p..."


time: 26.5 s (started: 2023-06-21 15:30:23 -07:00)


In [87]:
all_view.index.count().compute()

6989131

time: 4.73 s (started: 2023-06-21 16:18:28 -07:00)


In [112]:
!mv /p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet/global.json ../

time: 697 ms (started: 2023-06-21 16:45:58 -07:00)


In [7]:
log_dir2 = "/p/vast1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
all_view2 = pd.read_parquet(f"{log_dir2}")
all_view2

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
0,1,111084,0,314672,0,3,0.242405,0.242414,__xstat64,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.000000,0.000009,2424094,1571755611461267775,-4245390515581891276
1,2,111084,0,314672,0,3,0.242502,0.242732,fopen64,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964/2/run/Kimag...,0,0,0.000000,0.000230,2426168,7638324191939094055,-4245390515581891276
2,3,111084,0,314672,0,3,0.242851,0.242868,opendir,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.000000,0.000017,2428593,1571755611461267775,-4245390515581891276
3,4,111084,0,314672,0,3,0.242946,0.242956,__xstat64,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964/2,0,0,0.000000,0.000010,2429507,1571755611461267775,-4245390515581891276
4,5,111084,0,314672,0,3,0.243006,0.243016,__xstat64,0,...,mImgtbl,mImgtbl#lassen247#111084#35184372403504,/p/gpfs1/iopp/temp/montage/2854964,0,0,0.000000,0.000010,2430107,5347462497940582382,-4245390515581891276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6989126,2332457,41731,31,317888,0,2,1.604730,1.604747,fwrite,0,...,mAddMPI,mAddMPI#lassen246#41731#35184372406720,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,96920,0,5469.237793,0.000017,16047384,7686174940287124680,8238577613900339652
6989127,2332458,41731,31,317888,0,2,1.604971,1.604992,fwrite,0,...,mAddMPI,mAddMPI#lassen246#41731#35184372406720,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,96920,0,4359.911133,0.000021,16049814,7686174937900816169,8238577613900339652
6989128,2332459,41731,31,317888,0,3,1.605062,1.605241,fclose,0,...,mAddMPI,mAddMPI#lassen246#41731#35184372406720,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,0,0,0.000000,0.000179,16051513,7686174940287124680,8238577613900339652
6989129,2332460,41731,31,317888,0,3,1.605244,1.605379,fclose,0,...,mAddMPI,mAddMPI#lassen246#41731#35184372406720,/p/gpfs1/iopp/temp/montage/2854964/1/run/ngc33...,0,0,0.000000,0.000135,16053112,7686174937900816169,8238577613900339652


time: 18.3 s (started: 2023-07-02 22:52:23 -07:00)


### Data preparation

In [8]:
all_view_200k = all_view2.loc[:200_000]
all_view_1m = all_view2.loc[:1_000_000]
all_view_5m = all_view2.loc[:5_000_000]

time: 1.2 ms (started: 2023-07-02 22:52:41 -07:00)


### Iteration 

In [64]:
%%timeit -r 10 -n 1

size = 0
for row in all_view_200k.itertuples():
    size = size + getattr(row, 'size')
# size

704 ms ± 3.89 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 7.05 s (started: 2023-06-26 13:01:47 -07:00)


In [65]:
%%timeit -r 10 -n 1

size = 0
for row in all_view_1m.itertuples():
    size = size + getattr(row, 'size')
# size

3.51 s ± 36 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 35.1 s (started: 2023-06-26 13:02:29 -07:00)


In [66]:
%%timeit -r 10 -n 1

size = 0
for row in all_view_5m.itertuples():
    size = size + getattr(row, 'size')
# size

17.5 s ± 69.5 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 2min 54s (started: 2023-06-26 13:03:37 -07:00)


In [109]:
all_view_5m.index.count().compute()

6989131

time: 4.76 s (started: 2023-06-21 16:43:48 -07:00)


## Eval IV: Preprocessing - Query on raw vs. preprocessed data

In [9]:
def find_func(df):
    t0 = time()
    cond = (df['func_id'].str.contains('read')) | (df['func_id'].str.contains('write'))
    df[cond]
    t1 = time()
    df[~cond]
    t2 = time() - t1
    print(f"data,{t1-t0}")
    print(f"metadata,{t2}")


def find_func2(df):
    t0 = time()
    df[df['io_cat'].isin([1, 2])]
    t1 = time()
    df[df['io_cat'] == 3]
    t2 = time() - t1
    print(f"data,{t1-t0}")
    print(f"metadata,{t2}")

time: 1.53 ms (started: 2023-07-02 22:53:01 -07:00)


In [13]:
%%timeit -r 10 -n 1

find_func(all_view_200k)

data,0.17031574249267578
metadata,0.0032074451446533203
data,0.17721056938171387
metadata,0.004504203796386719
data,0.18410277366638184
metadata,0.0036156177520751953
data,0.18445205688476562
metadata,0.0038633346557617188
data,0.1849367618560791
metadata,0.00372314453125
data,0.18219470977783203
metadata,0.003223419189453125
data,0.18683505058288574
metadata,0.003323793411254883
data,0.18620991706848145
metadata,0.0038106441497802734
data,0.18181705474853516
metadata,0.0039055347442626953
data,0.1875
metadata,0.003746509552001953
186 ms ± 5 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 1.87 s (started: 2023-07-02 22:54:34 -07:00)


In [14]:
%%timeit -r 10 -n 1

find_func2(all_view_200k)

data,0.023721694946289062
metadata,0.004883766174316406
data,0.020141124725341797
metadata,0.0036706924438476562
data,0.01852893829345703
metadata,0.0034568309783935547
data,0.01730036735534668
metadata,0.003837108612060547
data,0.018039226531982422
metadata,0.0035114288330078125
data,0.01894664764404297
metadata,0.0033664703369140625
data,0.018018484115600586
metadata,0.0030608177185058594
data,0.017902374267578125
metadata,0.004578828811645508
data,0.020904541015625
metadata,0.0036962032318115234
data,0.02040266990661621
metadata,0.005652666091918945
23.4 ms ± 2.36 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 237 ms (started: 2023-07-02 22:54:51 -07:00)


In [15]:
%%timeit -r 10 -n 1

find_func(all_view_1m)

data,1.148200511932373
metadata,0.007668495178222656
data,0.8896410465240479
metadata,0.006735324859619141
data,0.9729976654052734
metadata,0.007321596145629883
data,0.8794040679931641
metadata,0.006827592849731445
data,0.9092130661010742
metadata,0.00825190544128418
data,0.9182817935943604
metadata,0.006508350372314453
data,0.8680040836334229
metadata,0.0063097476959228516
data,0.9648122787475586
metadata,0.006547451019287109
data,0.9262304306030273
metadata,0.006066560745239258
data,0.9672050476074219
metadata,0.0066301822662353516
951 ms ± 76.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 9.52 s (started: 2023-07-02 22:55:31 -07:00)


In [16]:
%%timeit -r 10 -n 1

find_func2(all_view_1m)

data,0.27687883377075195
metadata,0.007642507553100586
data,0.2116382122039795
metadata,0.0074939727783203125
data,0.1675729751586914
metadata,0.006834983825683594
data,0.2382824420928955
metadata,0.006891965866088867
data,0.1930549144744873
metadata,0.009273767471313477
data,0.18817567825317383
metadata,0.00609898567199707
data,0.17193913459777832
metadata,0.005898237228393555
data,0.1993696689605713
metadata,0.006345033645629883
data,0.20032048225402832
metadata,0.0063936710357666016
data,0.17356514930725098
metadata,0.009080886840820312
209 ms ± 32.1 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 2.1 s (started: 2023-07-02 22:55:40 -07:00)


In [17]:
%%timeit -r 10 -n 1

find_func(all_view_5m)

data,6.387158393859863
metadata,0.019707441329956055
data,5.443041801452637
metadata,0.019626379013061523
data,5.435396194458008
metadata,0.01870107650756836
data,5.405319929122925
metadata,0.01942920684814453
data,5.231921911239624
metadata,0.01784801483154297
data,5.3591015338897705
metadata,0.019825458526611328
data,5.207401275634766
metadata,0.017080068588256836
data,5.247430086135864
metadata,0.021250247955322266
data,5.606020927429199
metadata,0.01924872398376465
data,5.104741096496582
metadata,0.018557071685791016
5.46 s ± 344 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 54.6 s (started: 2023-07-02 22:55:52 -07:00)


In [18]:
%%timeit -r 10 -n 1

find_func2(all_view_5m)

data,1.6949751377105713
metadata,0.023679018020629883
data,1.7042655944824219
metadata,0.02284717559814453
data,1.4846436977386475
metadata,0.025501489639282227
data,1.9733805656433105
metadata,0.022776126861572266
data,1.7926025390625
metadata,0.02291131019592285
data,1.8427672386169434
metadata,0.019257783889770508
data,1.855384349822998
metadata,0.020868778228759766
data,1.9973492622375488
metadata,0.01880645751953125
data,2.007688045501709
metadata,0.02173757553100586
data,2.0111215114593506
metadata,0.02135467529296875
1.86 s ± 162 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 18.6 s (started: 2023-07-02 22:56:46 -07:00)


In [171]:
def calc_hlm_view(ddf, persist=False):
    t0 = time()
    hlm_view = ddf[(ddf['cat'] == 0) & (ddf['io_cat'].isin([1, 2, 3]))] \
        .groupby(['file_id']) \
        .agg({
            'index': 'count',
            'duration': sum,
            'size': sum,
        }) \
        .reset_index()
    t1 = time()
    if persist:
        hlm_view = hlm_view.persist()
        # wait(hlm_view)
    t2 = time()
    hlm_view['bw'] = hlm_view['size'] / hlm_view['duration']
    hlm_view['bw_norm'] = hlm_view['bw'] / hlm_view['bw'].sum()
    hlm_view['intensity'] = hlm_view['bw'] / hlm_view['index']
    hlm_view['dur_per'] = hlm_view['duration'] / hlm_view['duration'].max()
    final_view = hlm_view.compute()
    t3 = time()
    print(f"q1,{t1-t0}")
    print(f"persist,{t2-t1}")
    print(f"q2,{t3-t2}")

time: 2.31 ms (started: 2023-06-21 17:20:27 -07:00)


In [142]:
all_view_200k_ddf = dd.from_pandas(all_view_200k, npartitions=20)
all_view_1m_ddf = dd.from_pandas(all_view_1m, npartitions=100)
all_view_5m_ddf = dd.from_pandas(all_view_5m, npartitions=500)

time: 9.88 s (started: 2023-06-21 17:08:22 -07:00)


In [144]:
all_view_25m_ddf = dd.from_pandas(dd.concat([all_view_5m_ddf]*5).compute(), npartitions=2500)

time: 1min 32s (started: 2023-06-21 17:09:50 -07:00)


In [172]:
calc_hlm_view(all_view_200k_ddf)

q1,0.037634849548339844
persist,2.384185791015625e-07
q2,0.3123667240142822
time: 351 ms (started: 2023-06-21 17:20:31 -07:00)


In [173]:
calc_hlm_view(all_view_200k_ddf, True)

q1,0.02421855926513672
persist,0.014767646789550781
q2,0.31082749366760254
time: 351 ms (started: 2023-06-21 17:20:34 -07:00)


In [174]:
calc_hlm_view(all_view_1m_ddf)

q1,0.0230865478515625
persist,4.76837158203125e-07
q2,0.886458158493042
time: 911 ms (started: 2023-06-21 17:20:35 -07:00)


In [175]:
calc_hlm_view(all_view_1m_ddf, True)

q1,0.02448439598083496
persist,0.03750467300415039
q2,1.170438528060913
time: 1.23 s (started: 2023-06-21 17:20:37 -07:00)


In [176]:
calc_hlm_view(all_view_5m_ddf)

q1,0.025345563888549805
persist,4.76837158203125e-07
q2,3.9722607135772705
time: 4 s (started: 2023-06-21 17:20:40 -07:00)


In [177]:
calc_hlm_view(all_view_5m_ddf, True)

q1,0.024511098861694336
persist,0.14873647689819336
q2,3.4780001640319824
time: 3.65 s (started: 2023-06-21 17:20:44 -07:00)


In [178]:
calc_hlm_view(all_view_25m_ddf)

q1,0.06383562088012695
persist,4.76837158203125e-07
q2,23.824289083480835
time: 23.9 s (started: 2023-06-21 17:20:47 -07:00)


In [179]:
calc_hlm_view(all_view_25m_ddf, True)

q1,1.176560878753662
persist,0.6906352043151855
q2,23.874505758285522
time: 25.7 s (started: 2023-06-21 17:21:11 -07:00)
